In [2]:
source("../src/utils/custom_tools.R")
setup_environment("../src/utils")

Warning message:
“package ‘dplyr’ was built under R version 3.5.1”

In [4]:
impact <- read.table("../data/final_IMPACT_mutations_180508.txt",
                     sep = "\t", stringsAsFactors = FALSE, header = TRUE)